In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
import sys

sys.path.append('../../')

In [80]:
from src.indexing import get_multivector_retriever
from src.generation import QA_SYSTEM_PROMPT, QA_PROMPT, LLAMA_PROMPT_TEMPLATE, MIXTRAL_PROMPT_TEMPLATE
from src.generation import get_model, format_docs, get_rag_chain
from langchain_core.documents import Document

from src.ingestion import load_pdf

import os
import chromadb
import uuid

In [81]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [82]:
DATA_PATH = "../../data"
RAW_DOCS_PATH = os.path.join(DATA_PATH, "raw")
CHROMA_PATH = os.path.join(DATA_PATH, "chroma")
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
MODEL_NAME = "meta-llama/Llama-3-70b-chat-hf"

In [83]:
docs = [load_pdf(os.path.join(RAW_DOCS_PATH, f)) for f in os.listdir(RAW_DOCS_PATH) if ".pdf" in f]

In [84]:
# print(docs[1].page_content)

In [85]:
persistent_client = chromadb.PersistentClient(path='../../data/chroma')

In [ ]:
persistent_client.

In [11]:
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=400,
    separators=['\n\n\n', '\n\n', '\n', r'\.\s+', ' ', '']
)

parent_docs = parent_splitter.split_documents(docs)
parent_docs_ids = [str(uuid.uuid4()) for _ in parent_docs]
id_key = "parent_doc_id"

#### Generate questions for each document.

In [161]:
SYSTEM_PROMPT = """Write a list of 30 fact based simple questions in Arabic that can be answered using the document. \
The questions should be require specific numbers or information mentioned in the document for an answer. \
But the questions should include information present in the document itself. \
Each question should be understandable indepdent from any context. \
You SHOULD NOT use any numbers in the question (except for years). \
Write the questions in Arabic only and don't write the answers. \
Output the questions directly without an introduction."""

USER_MESSAGE = """Context: {context}"""

In [162]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(LLAMA_PROMPT_TEMPLATE.format(system_prompt=SYSTEM_PROMPT, user_message=USER_MESSAGE))

In [163]:
llm = get_model(MODEL_NAME)

In [164]:
question_chain = (
    prompt_template
    | llm
    | StrOutputParser()
)

In [165]:
print(question_chain.invoke(parent_docs[45].page_content))



Here are 30 fact-based simple questions in Arabic that can be answered using the document:

1. ما هو تاريخ إصدار البيان المالي السنوي؟
2. ما هو اسم الشركة القابضة التي تضم 4 شركات مملوكة لها؟
3. ما هو النسبة المئوية لحصة مجموعة تداول السعودية في رأس مال شركة تداول العقارية؟
4. ما هو هدف شركة سوق الكربون الطوعي الإقليمي؟
5. ما هو عدد الشركات المملوكة لمجموعة تداول السعودية؟
6. ما هو اسم شركة مركزي إيداع الأوراق المالية؟
7. ما هو النوع من الخدمات التي تقدمها شركة تداول للحلول المتقدمة؟
8. ما هو اسم الشركة القابضة التي تضم مجموعة تداول السعودية؟
9. ما هو النسبة المئوية لحصة مجموعة تداول السعودية في شركة سوق الكربون الطوعي الإقليمي؟
10. ما هو مجال عمل شركة تداول العقارية؟
11. ما هو اسم شركة مركزي مقاصة الأوراق المالية؟
12. ما هو هدف مجموعة تداول السعودية في دعم الشركات والقطاعات في المنطقة؟
13. ما هو النوع من الخدمات التي تقدمها شركة تداول السعودية؟
14. ما هو اسم الشركة التي تضم مجموعة تداول السعودية في مجال إدارة العقارات؟
15. ما هو النسبة المئوية لحصة مجموعة تداول السعودية في رأس مال ش

In [173]:
child_docs = question_chain.batch(parent_docs)

In [174]:
len(child_docs)

48

In [175]:
for i, d in enumerate(child_docs): print(i, '\n', d, '\n')

0 
 

Here are the 30 fact-based simple questions in Arabic:

1. ما هو عنوان التقرير الذي أصدرته وزارة المالية؟
2. فيما يلي، ما هو عنوان الفصل الذي يحتوي على المؤشرات الاقتصادية؟
3. ما هو رقم الصفحة الذي يبدأ فيه الملخص التنفيذي؟
4. ما هو عنوان الفصل الذي يحتوي على أداء المالية العامة؟
5. ما هو رقم الصفحة الذي يبدأ فيه الفصل الخاص بالإيرادات؟
6. ما هو عنوان الفصل الذي يحتوي على النفقات؟
7. فيما يلي، ما هو العام المالي الذي يcovered في التقرير؟
8. ما هو هدف رؤية المملكة؟
9. ما هو العام الذي يcovered في التقرير عن الميزانية العامة؟
10. ما هو عنوان الفصل الذي يحتوي على مقدمة التقرير؟
11. ما هو عنوان الفصل الذي يحتوي على أداء الميزانية الفعلي؟
12. ما هو رقم الصفحة الذي يبدأ فيه الفصل الخاص بالملخص التنفيذي؟
13. ما هو عنوان الفصل الذي يحتوي على المؤشرات الاقتصادية؟
14. ما هو عنوان الفصل الذي يحتوي على أداء المالية العامة؟
15. ما هو رقم الصفحة الذي يبدأ فيه الفصل الخاص بالإيرادات؟
16. ما هو عنوان الفصل الذي يحتوي على النفقات؟
17. ما هو العام المالي الذي يcovered في التقرير عن الميزانية العام

In [186]:
_child_docs = child_docs

In [227]:
child_docs = []
for i, doc in enumerate(_child_docs):
    _id = parent_docs_ids[i]
    _doc = Document(page_content=doc, metadata={id_key: _id, 'source': parent_docs[i].metadata['source']})
    child_docs.append(_doc)

In [228]:
child_docs[0].metadata

{'parent_doc_id': '853028cd-5503-4695-9051-4c56a867f4ab',
 'source': 'Actual Budget Report 2022'}

In [229]:
child_docs[0].metadata, parent_docs_ids[0]

({'parent_doc_id': '853028cd-5503-4695-9051-4c56a867f4ab',
  'source': 'Actual Budget Report 2022'},
 '853028cd-5503-4695-9051-4c56a867f4ab')

#### Save child documents using the collection name in interim documents

In [230]:
INTERIM_DATA_PATH = os.path.join(DATA_PATH, "interim")

In [72]:
FILE_NAME = f"PARENT_1200_400_QUESTIONS_{MODEL_NAME.replace('-', '_').replace('/', '_')}.pkl"
FILE_PATH = os.path.join(INTERIM_DATA_PATH, FILE_NAME)

In [73]:
print(FILE_PATH)

../../data\interim\PARENT_1200_400_QUESTIONS_meta_llama_Llama_3_70b_chat_hf.pkl


In [232]:
import pickle

pickle.dump(child_docs, open(FILE_PATH, 'wb'))

In [57]:
child_docs = pickle.load(open(FILE_PATH, 'rb'))

#### Create a multi vector retriever using the combined questions

In [62]:
COLLECTION_NAME = f"PARENT_1200_400_QUESTIONS_COMBINED_{MODEL_NAME.replace('-', '_').replace('/', '_')}"

In [63]:
embedding_model_name = "text-embedding-3-small"

In [64]:
save_path = 'D:\Ahmed\saudi-rag-project\data'

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_3016\3935730633.py:1: SyntaxWarning: invalid escape sequence '\A'
  save_path = 'D:\Ahmed\saudi-rag-project\data'


In [66]:
# persistent_client.delete_collection(COLLECTION_NAME)

In [67]:
retriever = get_multivector_retriever(persistent_client, EMBEDDING_MODEL_NAME, COLLECTION_NAME, save_path, parent_docs, parent_docs_ids, child_docs)

KeyboardInterrupt: 

In [240]:
len(child_docs), len(parent_docs)

(48, 48)

In [241]:
retriever.invoke("ما هو صافي الربح 2022؟")

[Document(page_content='• ارﺗﻔﻌﺖ اﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ ﺑﻨﺴﺒﺔ %16.6 ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي ﻟﺘﺼﻞ إﻟﻰ 644.3 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم\nم2022 ﻣﻘﺎرﻧﺔ ﺑ 552.5 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم اﻟﺴﺎﺑﻖ م2021 وﻳﻌﻮد ذﻟﻚ إﻟﻰ اﻻرﺗﻔﺎع ﻓﻲ ﺗﻜﺎﻟﻴﻒ اﻟﺮواﺗﺐ\nواﻟﻤﻨﺎﻓﻊ اﻟﻤﺘﻌﻠﻘﺔ ﺑﻬﺎ ﻣﺪﻓﻮﻋﺔ ﺑﺴﺒﺐ اﻟﺰﻳﺎدة ﻓﻲ أﻋﺪاد اﻟﻤﻮﻇﻔﻴﻦ وذﻟﻚ ﺣﺴﺐ ﺧﻄﺔ اﻟﻤﺠﻤﻮﻋﺔ.\n• اﻧﺨﻔﻀﺖ اﻷرﺑﺎح ﻗﺒﻞ اﻟﻔﻮاﺋﺪ واﻟﻀﺮاﺋﺐ واﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء (EBITDA) ﺑﻨﺴﺒﺔ %26.9 ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي ﻟﺘﺼﻞ\nإﻟﻰ 490.3 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم م2022 ﻣﻘﺎرﻧﺔ ﺑ 670.6 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم اﻟﺴﺎﺑﻖ م2021\nوﻳﻌﻮد ذﻟﻚ إﻟﻰ اﻧﺨﻔﺎض اﻹﻳﺮادات اﻟﺘﺸﻐﻴﻠﻴﺔ ﻟﻠﻤﺠﻤﻮﻋﺔ ﻣﻘﺎﺑﻞ اﻟﻨﻤﻮ ﻓﻲ اﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ ﻟﻠﻤﺠﻤﻮﻋﺔ.\n• اﻧﺨﻔﺾ إﺟﻤﺎﻟﻲ اﻟﺮﺑﺢ ﻟﻠﻤﺠﻤﻮﻋﺔ ﺑﻨﺴﺒﺔ %18.0 ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي ﻟﻴﺼﻞ إﻟﻰ 683.7 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ\nاﻟﻌﺎم م2022 ﻣﻘﺎرﻧﺔ ﺑ 834.3 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم اﻟﺴﺎﺑﻖ م2021 وﻳﻌﻮد ذﻟﻚ إﻟﻰ اﻻﻧﺨﻔﺎض ﻓﻲ\nاﻹﻳﺮادات اﻟﺘﺸﻐﻴﻠﻴﺔ ﻣﻘﺎﺑﻞ اﻟﻨﻤﻮ ﻓﻲ اﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ ﻟﻠﻤﺠﻤﻮﻋﺔ.', metadata={'source': 'Press Release - 2022 Results (Stock Market)', 'file_path': '../../data\\raw\\Press Release - 2022 Results (Stock Market).pdf'}),
 Document(

In [243]:
retriever.vectorstore.similarity_search("ما هو صافي الربح 2022؟", k=3)

[Document(page_content='\n\nHere are the 30 fact-based simple questions in Arabic:\n\n1. ما هو النسبة المئوية لارتفاع المصاريف التشغيلية عن أساس سنوي؟\n2. ما هو المبلغ الإجمالي للمصاريف التشغيلية في العام 2022؟\n3. ما هو المبلغ الإجمالي للمصاريف التشغيلية في العام 2021؟\n4. ما هو السبب الرئيسي لارتفاع المصاريف التشغيلية؟\n5. ما هو النسبة المئوية لانخفاض الأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء؟\n6. ما هو المبلغ الإجمالي للأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء في العام 2022؟\n7. ما هو المبلغ الإجمالي للأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء في العام 2021؟\n8. ما هو السبب الرئيسي لانخفاض الأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء؟\n9. ما هو النسبة المئوية لانخفاض إجمالي الربح للمجموعة؟\n10. ما هو المبلغ الإجمالي لربح المجموعة في العام 2022؟\n11. ما هو المبلغ الإجمالي لربح المجموعة في العام 2021؟\n12. ما هو السبب الرئيسي لانخفاض إجمالي الربح للمجموعة؟\n13. ما هو المبلغ الإجمالي للمصاريف التشغيلية في العام 2022 مقارنة بالعام 2021؟\n14. ما هو النسبة المئوية لا

In [244]:
retriever.vectorstore.similarity_search("ما هو اجمالي قيمة الزكاة للمجموعة 2022؟", k=3)

[Document(page_content='\n\nHere are the 30 fact-based simple questions in Arabic:\n\n1. ما هو النسبة المئوية لارتفاع المصاريف التشغيلية عن أساس سنوي؟\n2. ما هو المبلغ الإجمالي للمصاريف التشغيلية في العام 2022؟\n3. ما هو المبلغ الإجمالي للمصاريف التشغيلية في العام 2021؟\n4. ما هو السبب الرئيسي لارتفاع المصاريف التشغيلية؟\n5. ما هو النسبة المئوية لانخفاض الأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء؟\n6. ما هو المبلغ الإجمالي للأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء في العام 2022؟\n7. ما هو المبلغ الإجمالي للأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء في العام 2021؟\n8. ما هو السبب الرئيسي لانخفاض الأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء؟\n9. ما هو النسبة المئوية لانخفاض إجمالي الربح للمجموعة؟\n10. ما هو المبلغ الإجمالي لربح المجموعة في العام 2022؟\n11. ما هو المبلغ الإجمالي لربح المجموعة في العام 2021؟\n12. ما هو السبب الرئيسي لانخفاض إجمالي الربح للمجموعة؟\n13. ما هو المبلغ الإجمالي للمصاريف التشغيلية في العام 2022 مقارنة بالعام 2021؟\n14. ما هو النسبة المئوية لا

#### Create a multi vector store using split questions

In [252]:
import langid

def is_english(text):
    lang, _ = langid.classify(text)
    return lang == 'en'

In [253]:
child_docs = []
for i, doc in enumerate(_child_docs):
    _id = parent_docs_ids[i]
    for l in doc.split('\n'):
        if not is_english(l): 
            _doc = Document(page_content=l, metadata={id_key: _id, 'source': parent_docs[i].metadata['source']})
            child_docs.append(_doc)

In [254]:
len(child_docs)

1440

In [256]:
COLLECTION_NAME = f"PARENT_1200_400_QUESTIONS_SPLIT_{MODEL_NAME.replace('-', '_').replace('/', '_')}"

In [257]:
embedding_model_name = "text-embedding-3-small"

In [258]:
save_path = 'D:\Ahmed\saudi-rag-project\data'

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_19296\3935730633.py:1: SyntaxWarning: invalid escape sequence '\A'
  save_path = 'D:\Ahmed\saudi-rag-project\data'


In [259]:
retriever = get_multivector_retriever(persistent_client, EMBEDDING_MODEL_NAME, COLLECTION_NAME, save_path, parent_docs, parent_docs_ids, child_docs)

In [263]:
retriever.vectorstore.similarity_search_with_score("ما هي قيمة الزكاة للمجموعة 2022؟", k=3)

[(Document(page_content='18. ما هو إجمالي الزكاة لعام 2022؟', metadata={'parent_doc_id': '43d4b7e3-128d-41b1-abca-fb55fb26fa2a', 'source': 'Press Release - 2022 Results (Stock Market)'}),
  0.09632417559623718),
 (Document(page_content='24. ما هو المبلغ الإجمالي لربح المجموعة في العام 2022؟', metadata={'parent_doc_id': 'ab09a5ba-076a-46ea-a9c5-1509c7d9db4e', 'source': 'Press Release - 2022 Results (Stock Market)'}),
  0.11663046479225159),
 (Document(page_content='10. ما هو المبلغ الإجمالي لربح المجموعة في العام 2022؟', metadata={'parent_doc_id': 'ab09a5ba-076a-46ea-a9c5-1509c7d9db4e', 'source': 'Press Release - 2022 Results (Stock Market)'}),
  0.12297346442937851)]

In [264]:
retriever.invoke("ما هي قيمة الزكاة للمجموعة 2022؟")

[Document(page_content='ﺑﻴﺎن أرﺑﺎح اﻟﻨﺘﺎﺋﺞ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ ﻟﻠﻌﺎم م2022\n26 ﻓﺒﺮاﻳﺮ م2023\nﻧﻈﺮة ﻋﻠﻰ ﻗﺎﺋﻤﺔ اﻟﺮﺑﺢ واﻟﺨﺴﺎرة\nاﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﺘﻐﻴﺮ %\n)ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي(\nم2022 م2021 )ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي(\nاﻹﻳﺮادات اﻟﺘﺸﻐﻴﻠﻴﺔ 1,071.4 1,166.1 8.1% ▼\nاﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ 581.1 495.5 17.3% ▲\nاﻷرﺑﺎح ﻗﺒﻞ اﻟﻔﻮاﺋﺪ واﻟﻀﺮاﺋﺐ واﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء (EBITDA) 490.3 670.6 26.9% ▼\nﻫﺎﻣﺶ ﻗﺒﻞ اﻟﻔﻮاﺋﺪ واﻟﻀﺮاﺋﺐ واﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء (EBITDA) 46% 58%\nﻣﺼﺮوﻓﺎت اﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء 63.2 57.0 10.8% ▲\nاﻷرﺑﺎح ﻗﺒﻞ اﺣﺘﺴﺎب ﻣﺼﺎرﻳﻒ اﻟﻔﻮاﺋﺪ واﻟﺰﻛﺎة واﻟﻀﺮاﺋﺐ (EBIT) 427.1 613.6 30.4% ▼\nﻫﺎﻣﺶ اﻷرﺑﺎح ﻗﺒﻞ اﺣﺘﺴﺎب ﻣﺼﺎرﻳﻒ اﻟﻔﻮاﺋﺪ واﻟﺰﻛﺎة واﻟﻀﺮاﺋﺐ (EBIT) 40% 53%\nإﻳﺮادات اﻻﺳﺘﺜﻤﺎرات وإﻳﺮادات أﺧﺮى 75.1 43.6 72.1% ▲\nاﻟﺨﺴﺎرة ﻣﻦ اﻟﺸﺮﻛﺎت اﻟﺰﻣﻴﻠﺔ (9.9) (3.3) 202.5% ▲\nاﻟﺰﻛﺎة 67.7 66.2 2.2% ▲\nﺻﺎﻓﻲ اﻟﺮﺑﺢ ﺑﻌﺪ اﻟﺰﻛﺎة 424.6 587.7 27.8% ▼\nﻫﺎﻣﺶ ﺻﺎﻓﻲ اﻟﺮﺑﺢ ﺑﻌﺪ اﻟﺰﻛﺎة 40% 50%\nرﺑﺤﻴﺔ اﻟﺴﻬﻢ )رﻳﺎل ﺳﻌﻮدي( 3.54 4.90\nﻧﻈﺮة ﻋﻠﻰ اﻟﻤﺮﻛﺰ اﻟﻤﺎﻟﻲ\nاﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﺘﻐﻴﺮ %\n)ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي(\nم2022 م2021 )ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي(\nاﻟﻨﻘﺪ وﻣﺎ ﻳﻌﺎدﻟﻪ 2,118.8 86.2 2358

#### Generate a summary for each document

In [34]:
SYSTEM_PROMPT = """You are an expert summary writer. \
Summarize the following document. \
Also create a title for what this document talks about. \
The summary should be in Arabic only."""

USER_MESSAGE = """Document: {context}"""

In [35]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(LLAMA_PROMPT_TEMPLATE.format(system_prompt=SYSTEM_PROMPT, user_message=USER_MESSAGE))

In [36]:
llm = get_model("meta-llama/Llama-3-70b-chat-hf")

In [37]:
summary_chain = (
    prompt_template
    | llm
    | StrOutputParser()
)

In [38]:
print(summary_chain.invoke(parent_docs[4].page_content))



**Title:** نتائج الميزانية لعام 2022: فائض مالي ونمو اقتصادي

**Summary:**

تجاوزت الميزانية لعام 2022 التوقعات بفائض مالي بلغ 104 مليار ريال، أي ما يعادل 2.5% من الناتج المحلي الإجمالي. وسيتوجه هذا الفائض إلى تعزيز الاحتياطي الحكومي ودعم الصناديق الوطنية، بالإضافة إلى إمكانية تعجيل تنفيذ بعض البرامج والمشاريع الاستراتيجية. بلغ رصيد الدين العام حوالي 990 مليار ريال، في حين بلغ رصيد الاحتياطيات الحكومية حوالي 318 مليار ريال. سجل معدل نمو الناتج المحلي الحقيقي لعام 2022 نمواً بلغ 8.7%، أعلى من التوقعات، بفضل نمو الأنشطة النفطية بنسبة 15.4% والأنشطة غير النفطية بنسبة 5.4%.


In [39]:
print(summary_chain.invoke(parent_docs[43].page_content))



**Title:** إيرادات قطاع خدمات المعلومات والتكنولوجيا في المملكة العربية السعودية

**Summary:** بلغت إيرادات قطاع خدمات المعلومات والتكنولوجيا في المملكة العربية السعودية 100.2 مليون ريال سعودي في عام 2022، بزيادة نسبتها 7.4% عن العام السابق. كما انخفضت إيرادات قطاع ما بعد التداول بنسبة 3.1% عن العام السابق، لتصل إلى 564.7 مليون ريال سعودي في عام 2022. ويعود ذلك إلى انخفاض أنشطة التداول بنسبة 23.0%، مما أدى إلى خفض رسوم خدمات ما بعد التداول.


In [40]:
summary_docs = summary_chain.batch(parent_docs)

In [41]:
parent_docs_ids = [str(uuid.uuid4()) for _ in parent_docs]
id_key = "parent_doc_id"

In [42]:
child_docs = []
for i, doc in enumerate(summary_docs):
    _id = parent_docs_ids[i]
    _doc = Document(page_content=doc, metadata={id_key: _id, 'source': parent_docs[i].metadata['source']})
    child_docs.append(_doc)

In [43]:
child_docs[0].metadata

{'parent_doc_id': 'da2b3f04-3aa3-4bef-80cd-ad259ce9d750',
 'source': 'Actual Budget Report 2022'}

In [44]:
child_docs[0].metadata, parent_docs_ids[0]

({'parent_doc_id': 'da2b3f04-3aa3-4bef-80cd-ad259ce9d750',
  'source': 'Actual Budget Report 2022'},
 'da2b3f04-3aa3-4bef-80cd-ad259ce9d750')

#### Save child documents using the collection name in interim documents

In [45]:
INTERIM_DATA_PATH = os.path.join(DATA_PATH, "interim")

In [46]:
FILE_NAME = f"PARENT_1200_400_SUMMARIES_{MODEL_NAME.replace('-', '_').replace('/', '_')}.pkl"
FILE_PATH = os.path.join(INTERIM_DATA_PATH, FILE_NAME)

In [47]:
import pickle

pickle.dump(child_docs, open(FILE_PATH, 'wb'))

#### Create a multi vector retriever using the combined questions

In [48]:
COLLECTION_NAME = f"PARENT_1200_400_SUMMARIES_{MODEL_NAME.replace('-', '_').replace('/', '_')}"

In [50]:
save_path = 'D:\Ahmed\saudi-rag-project\data'

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_3016\3935730633.py:1: SyntaxWarning: invalid escape sequence '\A'
  save_path = 'D:\Ahmed\saudi-rag-project\data'


In [51]:
# persistent_client.delete_collection(COLLECTION_NAME)

In [52]:
retriever = get_multivector_retriever(persistent_client, EMBEDDING_MODEL_NAME, COLLECTION_NAME, save_path, parent_docs, parent_docs_ids, child_docs)

In [53]:
len(child_docs), len(parent_docs)

(48, 48)

In [54]:
retriever.invoke("ما هو صافي الربح 2022؟")

[Document(page_content='• ارﺗﻔﻌﺖ اﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ ﺑﻨﺴﺒﺔ %16.6 ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي ﻟﺘﺼﻞ إﻟﻰ 644.3 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم\nم2022 ﻣﻘﺎرﻧﺔ ﺑ 552.5 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم اﻟﺴﺎﺑﻖ م2021 وﻳﻌﻮد ذﻟﻚ إﻟﻰ اﻻرﺗﻔﺎع ﻓﻲ ﺗﻜﺎﻟﻴﻒ اﻟﺮواﺗﺐ\nواﻟﻤﻨﺎﻓﻊ اﻟﻤﺘﻌﻠﻘﺔ ﺑﻬﺎ ﻣﺪﻓﻮﻋﺔ ﺑﺴﺒﺐ اﻟﺰﻳﺎدة ﻓﻲ أﻋﺪاد اﻟﻤﻮﻇﻔﻴﻦ وذﻟﻚ ﺣﺴﺐ ﺧﻄﺔ اﻟﻤﺠﻤﻮﻋﺔ.\n• اﻧﺨﻔﻀﺖ اﻷرﺑﺎح ﻗﺒﻞ اﻟﻔﻮاﺋﺪ واﻟﻀﺮاﺋﺐ واﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء (EBITDA) ﺑﻨﺴﺒﺔ %26.9 ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي ﻟﺘﺼﻞ\nإﻟﻰ 490.3 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم م2022 ﻣﻘﺎرﻧﺔ ﺑ 670.6 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم اﻟﺴﺎﺑﻖ م2021\nوﻳﻌﻮد ذﻟﻚ إﻟﻰ اﻧﺨﻔﺎض اﻹﻳﺮادات اﻟﺘﺸﻐﻴﻠﻴﺔ ﻟﻠﻤﺠﻤﻮﻋﺔ ﻣﻘﺎﺑﻞ اﻟﻨﻤﻮ ﻓﻲ اﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ ﻟﻠﻤﺠﻤﻮﻋﺔ.\n• اﻧﺨﻔﺾ إﺟﻤﺎﻟﻲ اﻟﺮﺑﺢ ﻟﻠﻤﺠﻤﻮﻋﺔ ﺑﻨﺴﺒﺔ %18.0 ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي ﻟﻴﺼﻞ إﻟﻰ 683.7 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ\nاﻟﻌﺎم م2022 ﻣﻘﺎرﻧﺔ ﺑ 834.3 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي ﻓﻲ اﻟﻌﺎم اﻟﺴﺎﺑﻖ م2021 وﻳﻌﻮد ذﻟﻚ إﻟﻰ اﻻﻧﺨﻔﺎض ﻓﻲ\nاﻹﻳﺮادات اﻟﺘﺸﻐﻴﻠﻴﺔ ﻣﻘﺎﺑﻞ اﻟﻨﻤﻮ ﻓﻲ اﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ ﻟﻠﻤﺠﻤﻮﻋﺔ.', metadata={'source': 'Press Release - 2022 Results (Stock Market)', 'file_path': '../../data\\raw\\Press Release - 2022 Results (Stock Market).pdf'}),
 Document(

In [55]:
retriever.vectorstore.similarity_search_with_score("ما هو صافي الربح 2022؟")

[(Document(page_content='\n\n**Title:** نتائج العام 2022: ارتفاع المصاريف التشغيلية وتراجع الأرباح\n\n**Summary:** \n\nأظهرت نتائج العام 2022 ارتفاعاً في المصاريف التشغيلية بنسبة 16.6% لتصل إلى 644.3 مليون ريال سعودي مقارنةً بـ 552.5 مليون ريال سعودي في العام السابق 2021، وذلك بسبب الزيادة في أعداد الموظفين. كما انخفضت الأرباح قبل الفوائد والضرائب والاستهلاك والإطفاء (EBITDA) بنسبة 26.9% لتصل إلى 490.3 مليون ريال سعودي مقارنةً بـ 670.6 مليون ريال سعودي في العام السابق 2021، وذلك بسبب انخفاض الإيرادات التشغيلية للمجموعة مقابلاً للنمو في المصاريف التشغيلية. كما انخفض إجمالي الربح للمجموعة بنسبة 18.0% لتصل إلى 683.7 مليون ريال سعودي مقارنةً بـ 834.3 مليون ريال سعودي في العام السابق 2021، وذلك بسبب انخفاض الإيرادات التشغيلية مقابلاً للنمو في المصاريف التشغيلية.', metadata={'parent_doc_id': '5b718c6a-02f1-472c-a31c-9a37e19af158', 'source': 'Press Release - 2022 Results (Stock Market)'}),
  0.2175685572606833),
 (Document(page_content='\n\n**Title:** تقرير الميزانية الفعلي لعام 2022: الفوائض